<a href="https://colab.research.google.com/github/alirezash97/Titanic/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [299]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


trainset = pd.read_csv("/content/drive/My Drive/titanic project/train.csv")
testset = pd.read_csv("/content/drive/My Drive/titanic project/test.csv")
testset.head()





,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
def preprocessing(table):
    table = table.rename(columns = {'Pclass' : 'TicketClass'})
    table = table.drop(['Name','Ticket','Cabin'],axis =1)
    table['Sex'].replace({'male': 1, 'female':2 }, inplace=True)
    table['Age'].replace({float('NaN'): 29 }, inplace=True)
    table['Embarked'].replace({float('NaN'): 0,'S': 1 , 'C': 2, 'Q': 3}, inplace=True)
    return table

In [0]:
trainset = preprocessing(trainset)
testset = preprocessing(testset)

In [227]:
print(testset.head())

   PassengerId  TicketClass  Sex   Age  SibSp  Parch     Fare  Embarked
0          892            3    1  34.5      0      0   7.8292         3
1          893            3    2  47.0      1      0   7.0000         1
2          894            2    1  62.0      0      0   9.6875         3
3          895            3    1  27.0      0      0   8.6625         1
4          896            3    2  22.0      1      1  12.2875         1


In [302]:
trainset = trainset[['PassengerId','TicketClass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived']]
trainset.head()

,PassengerId,TicketClass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,1,3,1,22.0,1,0,7.2500,1,0
1,2,1,2,38.0,1,0,71.2833,2,1
2,3,3,2,26.0,0,0,7.9250,1,1
3,4,1,2,35.0,1,0,53.1000,1,1
4,5,3,1,35.0,0,0,8.0500,1,0


In [211]:
print(trainset.shape)

(891, 9)


In [0]:
X_train = trainset.iloc[:,:8]
y_train = trainset.iloc[:,8]

In [304]:
from numpy import loadtxt
from keras.models import Sequential, Model
from keras.layers import Input, Dense

model = Sequential()
model.add(Dense(64, input_dim=8, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128,  activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_119 (Dense)            (None, 64)                576       
_________________________________________________________________
dense_120 (Dense)            (None, 256)               16640     
_________________________________________________________________
dense_121 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_122 (Dense)            (None, 1)                 129       
Total params: 50,241
Trainable params: 50,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [306]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
891/891 [==============================] - 2s 2ms/step - loss: 5.6547 - acc: 0.6061
Epoch 2/100
891/891 [==============================] - 0s 71us/step - loss: 4.1851 - acc: 0.5993
Epoch 3/100
891/891 [==============================] - 0s 76us/step - loss: 3.6306 - acc: 0.5724
Epoch 4/100
891/891 [==============================] - 0s 73us/step - loss: 3.1261 - acc: 0.6016
Epoch 5/100
891/891 [==============================] - 0s 75us/step - loss: 1.1655 - acc: 0.6218
Epoch 6/100
891/891 [==============================] - 0s 75us/step - loss: 0.8482 - acc: 0.5993
Epoch 7/100
891/891 [==============================] - 0s 76us/step - loss: 1.1361 - acc: 0.5769
Epoch 8/100
891/891 [==============================] - 0s 74us/step - loss: 0.6848 - acc: 0.6667
Epoch 9/100
891/891 [==============================] - 0s 74us/step - loss: 0.7488 - acc: 0.6487
Epoch 10/100
891/891 [==============================] - 0s 71us/step - loss: 0.6818 - acc: 0.6611
Epoch 11/100
891/891 [========

In [0]:
predictions = model.predict(testset, batch_size=32)

In [308]:
predictions = predictions.flatten() 
print(predictions[:10])
for index,i in enumerate(predictions):
  if i > 0.5:
    predictions[index] = 1
  else:
    predictions[index] = 0

print(predictions[:10])

[0.03292763 0.02193117 0.01709634 0.03734237 0.13082281 0.09168661
 0.11587337 0.2604177  0.18613842 0.13057068]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
passid = np.array(testset.iloc[:, 0:1])
passid = passid.flatten()


In [0]:
predictions = pd.DataFrame({'PassengerId': passid, 'Survived': predictions}, index=[i for i in range(418)])

In [0]:
predictions.to_csv('Submission.csv', header=False, index=False)